In [5]:
import os
import torch
import pickle
import numpy as np
%pip install rembg
from rembg import remove
import pandas as pd
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm

print("✅ Librerías cargadas. ¡A por el reto de Inditex!")

/home/aleixbertranandreu/Documents/HackUDC_2026/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
✅ Librerías cargadas. ¡A por el reto de Inditex!


In [6]:
# 1. Preparar la tarjeta gráfica
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Usando dispositivo: {DEVICE.upper()}")

# 2. Cargar el modelo CLIP (el mismo que usó tu compañero)
print("Cargando modelo CLIP...")
modelo_id = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(modelo_id).to(DEVICE)
processor = CLIPProcessor.from_pretrained(modelo_id)
model.eval() # Modo inferencia (más rápido)

# 3. Cargar la base de datos vectorial (el archivo de Google Drive)
# ⚠️ IMPORTANTE: Pon la ruta correcta donde hayas guardado el .pkl
RUTA_PKL = "../data/embeddings/catalog_embeddings.pkl" 

with open(RUTA_PKL, "rb") as f:
    catalogo_db = pickle.load(f)

# Extraemos las claves (IDs) y la matriz de vectores para hacer matemáticas rápidas
ids_catalogo = list(catalogo_db.keys())
matriz_catalogo = np.vstack(list(catalogo_db.values()))

print(f"✅ Catálogo listo: {len(ids_catalogo)} prendas cargadas.")
print(f"📊 Forma de la matriz: {matriz_catalogo.shape}")

🚀 Usando dispositivo: CUDA
Cargando modelo CLIP...


Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

CLIPModel LOAD REPORT from: openai/clip-vit-base-patch32
Key                                  | Status     |  | 
-------------------------------------+------------+--+-
text_model.embeddings.position_ids   | UNEXPECTED |  | 
vision_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Catálogo listo: 27687 prendas cargadas.
📊 Forma de la matriz: (27687, 512)


In [7]:
from rembg import remove

def obtener_top_15_prendas(ruta_imagen, model, processor, matriz_catalogo, ids_catalogo, device="cuda"):
    try:
        # 1. Abrir la foto original
        imagen_original = Image.open(ruta_imagen).convert("RGB")
        
        # 🔥 EL TRUCO MÁGICO: Quitar el fondo 🔥
        # Esto quita la playa, la calle o la habitación y deja solo a la modelo
        imagen_limpia = remove(imagen_original)
        
        # CLIP prefiere fondos blancos (RGB) en lugar de transparentes (RGBA)
        # Así que la convertimos a RGB puro con fondo blanco
        imagen_final = Image.new("RGB", imagen_limpia.size, (255, 255, 255))
        imagen_final.paste(imagen_limpia, mask=imagen_limpia.split()[3]) # Usamos el canal Alpha como máscara
        
        with torch.no_grad():
            # 2. Pasamos la foto LIMPIA por CLIP
            inputs = processor(images=imagen_final, return_tensors="pt").to(device)
            vision_outputs = model.vision_model(pixel_values=inputs['pixel_values'])
            vector_modelo = model.visual_projection(vision_outputs.pooler_output)
            
            # 3. Normalizamos
            vector_modelo = vector_modelo / vector_modelo.norm(dim=-1, keepdim=True)
            vector_modelo = vector_modelo.cpu().squeeze().numpy()
            
        # 4. Calcular la similitud con el catálogo de tu compañero
        similitudes = np.dot(matriz_catalogo, vector_modelo)
        
        # 5. Coger los 15 con mayor puntuación
        top_15_indices = np.argsort(similitudes)[-15:][::-1]
        top_15_ids = [ids_catalogo[i] for i in top_15_indices]
        
        return top_15_ids
        
    except Exception as e:
        print(f"❌ Error con {ruta_imagen}: {e}")
        return []

In [16]:
foto_prueba = "../data/images/bundles/B_00a5b40f891c.jpg"

print(f"🔍 Analizando el outfit de la foto: {foto_prueba}")
resultados_prueba = obtener_top_15_prendas(foto_prueba, model, processor, matriz_catalogo, ids_catalogo, DEVICE)

print("\n🏆 LOS 15 IDs DE ROPA ENCONTRADOS:")
for i, id_ropa in enumerate(resultados_prueba, 1):
    print(f" {i}. {id_ropa}")

🔍 Analizando el outfit de la foto: ../data/images/bundles/B_00a5b40f891c.jpg

🏆 LOS 15 IDs DE ROPA ENCONTRADOS:
 1. I_476e19c1215a
 2. I_0a38d86df6be
 3. I_ca4149195342
 4. I_01f7658ce82d
 5. I_705a27210e2f
 6. I_73b2ad546686
 7. I_98a981fbe89e
 8. I_fd2f458dd09c
 9. I_2aef3ecb663a
 10. I_54b60f034dd4
 11. I_9fb545d08813
 12. I_65ec701ccf76
 13. I_18a90e72b83b
 14. I_081e0d86f98f
 15. I_4687bd8ba7d9


In [8]:
import pandas as pd
from tqdm import tqdm
import os

# --- CONFIGURACIÓN ---
# La carpeta donde tienes guardadas las fotos de los modelos (bundles) del Test
ruta_carpeta_test = "../data/images/bundles/" 
ruta_output_csv = "submission_inditex.csv"

# Cogemos todas las fotos que haya en esa carpeta
archivos_test = [f for f in os.listdir(ruta_carpeta_test) if f.endswith('.jpg')]
resultados_submission = []

print(f"🚀 Arrancando motores: Procesando {len(archivos_test)} fotos para el Test Final...")

# --- EL BUCLE MÁGICO ---
for archivo in tqdm(archivos_test, desc="Analizando outfits"):
    bundle_id = archivo.replace(".jpg", "") # Le quitamos el .jpg para tener el ID limpio
    ruta_foto = os.path.join(ruta_carpeta_test, archivo)
    
    # 1. Sacamos las 15 predicciones con tu función
    top_15_ids = obtener_top_15_prendas(ruta_foto, model, processor, matriz_catalogo, ids_catalogo, DEVICE)
    
    # 2. Las guardamos en el formato exacto que pide Inditex
    # (Una fila por cada predicción)
    for product_id in top_15_ids:
        resultados_submission.append({
            'bundle_asset_id': bundle_id,
            'product_asset_id': product_id
        })

# --- GUARDAR EL ARCHIVO ---
df_submission = pd.DataFrame(resultados_submission)
df_submission.to_csv(ruta_output_csv, index=False)

print("\n" + "="*50)
print(f"🎉 ¡RETO COMPLETADO! Archivo '{ruta_output_csv}' generado con éxito.")
print(f"Filas totales en el CSV: {len(df_submission)}")
print("="*50)

🚀 Arrancando motores: Procesando 2331 fotos para el Test Final...


Analizando outfits:   0%|          | 0/2331 [00:00<?, ?it/s]Downloading data from 'https://github.com/danielgatis/rembg/releases/download/v0.0.0/u2net.onnx' to file '/home/aleixbertranandreu/.u2net/u2net.onnx'.


  0%|                                               | 0.00/176M [00:00<?, ?B/s]

Analizando outfits: 100%|██████████| 2331/2331 [59:38<00:00,  1.54s/it] 


🎉 ¡RETO COMPLETADO! Archivo 'submission_inditex.csv' generado con éxito.
Filas totales en el CSV: 34965
